In [34]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.python.data import Dataset
from sklearn import metrics

# Read data

In [4]:
data = pd.read_csv("data/dengue_features_train.csv")

print data.shape
data.columns

(1456, 24)


Index([u'city', u'year', u'weekofyear', u'week_start_date', u'ndvi_ne',
       u'ndvi_nw', u'ndvi_se', u'ndvi_sw', u'precipitation_amt_mm',
       u'reanalysis_air_temp_k', u'reanalysis_avg_temp_k',
       u'reanalysis_dew_point_temp_k', u'reanalysis_max_air_temp_k',
       u'reanalysis_min_air_temp_k', u'reanalysis_precip_amt_kg_per_m2',
       u'reanalysis_relative_humidity_percent',
       u'reanalysis_sat_precip_amt_mm',
       u'reanalysis_specific_humidity_g_per_kg', u'reanalysis_tdtr_k',
       u'station_avg_temp_c', u'station_diur_temp_rng_c',
       u'station_max_temp_c', u'station_min_temp_c', u'station_precip_mm'],
      dtype='object')

In [7]:
data.describe()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
count,1456.000000,1456.000000,1262.000000,1404.000000,1434.000000,1434.000000,1443.000000,1446.000000,1446.000000,1446.000000,...,1446.000000,1446.000000,1443.000000,1446.000000,1446.000000,1413.000000,1413.000000,1436.000000,1442.000000,1434.000000
mean,2001.031593,26.503434,0.142294,0.130553,0.203783,0.202305,45.760388,298.701852,299.225578,295.246356,...,40.151819,82.161959,45.760388,16.746427,4.903754,27.185783,8.059328,32.452437,22.102150,39.326360
std,5.408314,15.019437,0.140531,0.119999,0.073860,0.083903,43.715537,1.362420,1.261715,1.527810,...,43.434399,7.153897,43.715537,1.542494,3.546445,1.292347,2.128568,1.959318,1.574066,47.455314
min,1990.000000,1.000000,-0.406250,-0.456100,-0.015533,-0.063457,0.000000,294.635714,294.892857,289.642857,...,0.000000,57.787143,0.000000,11.715714,1.357143,21.400000,4.528571,26.700000,14.700000,0.000000
25%,1997.000000,13.750000,0.044950,0.049217,0.155087,0.144209,9.800000,297.658929,298.257143,294.118929,...,13.055000,77.177143,9.800000,15.557143,2.328571,26.300000,6.514286,31.100000,21.100000,8.700000
50%,2002.000000,26.500000,0.128817,0.121429,0.196050,0.189450,38.340000,298.646429,299.289286,295.640714,...,27.245000,80.301429,38.340000,17.087143,2.857143,27.414286,7.300000,32.800000,22.200000,23.850000
75%,2005.000000,39.250000,0.248483,0.216600,0.248846,0.246982,70.235000,299.833571,300.207143,296.460000,...,52.200000,86.357857,70.235000,17.978214,7.625000,28.157143,9.566667,33.900000,23.300000,53.900000
max,2010.000000,53.000000,0.508357,0.454429,0.538314,0.546017,390.600000,302.200000,302.928571,298.450000,...,570.500000,98.610000,390.600000,20.461429,16.028571,30.800000,15.800000,42.200000,25.600000,543.300000


# Train model

In [11]:
data[["reanalysis_relative_humidity_percent"]][:10]

,reanalysis_relative_humidity_percent
0,73.365714
1,77.368571
2,82.052857
3,80.337143
4,80.460000
5,79.891429
6,82.000000
7,83.375714
8,82.768571
9,81.281429


In [53]:
data["week_start_date"].value_counts().keys()

Index([u'2006-09-03', u'2001-11-26', u'2003-03-26', u'2006-02-26',
       u'2007-03-26', u'2001-06-25', u'2003-07-30', u'2002-11-12',
       u'2007-10-01', u'2004-07-29',
       ...
       u'1992-06-24', u'1992-12-23', u'2008-09-02', u'1993-04-23',
       u'2010-05-07', u'1990-04-30', u'1990-10-15', u'1992-09-23',
       u'1997-04-16', u'1992-03-18'],
      dtype='object', length=1049)

In [72]:
values = {'city': "notknown", 'year': "notknown", 'weekofyear': "notknown", 'week_start_date': "notknown",
          'ndvi_ne': 0, 'ndvi_nw': 0, 'ndvi_se': 0, 'ndvi_sw': 0,
          'precipitation_amt_mm': 0, 'reanalysis_air_temp_k': 0, 'reanalysis_avg_temp_k': 0,
       'reanalysis_dew_point_temp_k': 0, 'reanalysis_max_air_temp_k': 0,
       'reanalysis_min_air_temp_k': 0, 'reanalysis_precip_amt_kg_per_m2': 0,
       'reanalysis_relative_humidity_percent': 0,
       'reanalysis_sat_precip_amt_mm': 0,
       'reanalysis_specific_humidity_g_per_kg': 0, 'reanalysis_tdtr_k': 0,
       'station_avg_temp_c': 0, 'station_diur_temp_rng_c': 0,
       'station_max_temp_c': 0, 'station_min_temp_c':0, 'station_precip_mm':0}
data = data.fillna(values)

In [74]:
# Define the input feature: reanalysis_relative_humidity_percent.
input_feature = data[[u'city', u'year', u'weekofyear', u'week_start_date', u'ndvi_ne',
       u'ndvi_nw', u'ndvi_se', u'ndvi_sw', u'precipitation_amt_mm',
       u'reanalysis_air_temp_k', u'reanalysis_avg_temp_k',
       u'reanalysis_dew_point_temp_k', u'reanalysis_max_air_temp_k',
       u'reanalysis_min_air_temp_k', u'reanalysis_precip_amt_kg_per_m2',
       u'reanalysis_relative_humidity_percent',
       u'reanalysis_sat_precip_amt_mm',
       u'reanalysis_specific_humidity_g_per_kg', u'reanalysis_tdtr_k',
       u'station_avg_temp_c', u'station_diur_temp_rng_c',
       u'station_max_temp_c', u'station_min_temp_c', u'station_precip_mm']]

# Configure a numeric feature column for reanalysis_relative_humidity_percent.
feature_columns = [tf.feature_column.categorical_column_with_vocabulary_list("city", data["city"].value_counts().keys()),
                   tf.feature_column.categorical_column_with_vocabulary_list("year", data["year"].value_counts().keys()),
                   tf.feature_column.categorical_column_with_vocabulary_list("weekofyear", data["weekofyear"].value_counts().keys()),
                   tf.feature_column.categorical_column_with_vocabulary_list("week_start_date", data["week_start_date"].value_counts().keys()),
                   tf.feature_column.numeric_column("ndvi_ne"),
                   tf.feature_column.numeric_column("ndvi_nw"),
                   tf.feature_column.numeric_column("ndvi_se"),
                   tf.feature_column.numeric_column("ndvi_sw"),
                   tf.feature_column.numeric_column("precipitation_amt_mm"),
                   tf.feature_column.numeric_column("reanalysis_air_temp_k"),
                   tf.feature_column.numeric_column("reanalysis_avg_temp_k"),
                   tf.feature_column.numeric_column("reanalysis_dew_point_temp_k"),
                   tf.feature_column.numeric_column("reanalysis_max_air_temp_k"),
                   tf.feature_column.numeric_column("reanalysis_min_air_temp_k"),
                   tf.feature_column.numeric_column("reanalysis_precip_amt_kg_per_m2"),
                   tf.feature_column.numeric_column("reanalysis_relative_humidity_percent"),
                   tf.feature_column.numeric_column("reanalysis_sat_precip_amt_mm"),
                   tf.feature_column.numeric_column("reanalysis_specific_humidity_g_per_kg"),
                   tf.feature_column.numeric_column("reanalysis_tdtr_k"),
                   tf.feature_column.numeric_column("station_avg_temp_c"),
                   tf.feature_column.numeric_column("station_diur_temp_rng_c"),
                   tf.feature_column.numeric_column("station_max_temp_c"),
                   tf.feature_column.numeric_column("station_min_temp_c"),
                   tf.feature_column.numeric_column("station_precip_mm")
                  ]

In [75]:
labels = pd.read_csv("data/dengue_labels_train.csv")

print labels.shape
labels.columns

(1456, 4)


Index([u'city', u'year', u'weekofyear', u'total_cases'], dtype='object')

In [76]:
# Define the label.
targets = labels["total_cases"]

In [83]:
def input_function(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model of one feature.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
  
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features, targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(buffer_size=10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [94]:
# Use gradient descent as the optimizer for training the model.
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0000000000001)
optimizer=tf.contrib.estimator.clip_gradients_by_norm(optimizer, 5.0)

# Configure the linear regression model with our feature columns and optimizer.
# Set a learning rate of 0.0000001 for Gradient Descent.
linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=optimizer
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c22026a50>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/qb/4n9_n15j4hn9x10gxyrppb280000gn/T/tmpZUFM3s', '_save_summary_steps': 100}


In [95]:
model = linear_regressor.train(
    input_fn = lambda:input_function(input_feature, targets),
    steps=100
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/qb/4n9_n15j4hn9x10gxyrppb280000gn/T/tmpZUFM3s/model.ckpt.
INFO:tensorflow:loss = 961.0, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /var/folders/qb/4n9_n15j4hn9x10gxyrppb280000gn/T/tmpZUFM3s/model.ckpt.
INFO:tensorflow:Loss for final step: 9.728634e-16.


In [96]:
# Create an input function for predictions.
# Note: Since we're making just one prediction for each example, we don't 
# need to repeat or shuffle the data here.
prediction_input_fn =lambda: input_function(input_feature, targets, num_epochs=1, shuffle=False)

# Call predict() on the linear_regressor to make predictions.
predictions = linear_regressor.predict(input_fn=prediction_input_fn)

# Format predictions as a NumPy array, so we can calculate error metrics.
predictions = np.array([item['predictions'][0] for item in predictions])

# Print Mean Squared Error and Root Mean Squared Error.
mean_squared_error = metrics.mean_squared_error(predictions, targets)
root_mean_squared_error = math.sqrt(mean_squared_error)
print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/qb/4n9_n15j4hn9x10gxyrppb280000gn/T/tmpZUFM3s/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


NameError: name 'math' is not defined

In [32]:
min_house_value = targets.min()
max_house_value = targets.max()
min_max_difference = max_house_value - min_house_value

print("Min. Median House Value: %0.3f" % min_house_value)
print("Max. Median House Value: %0.3f" % max_house_value)
print("Difference between Min. and Max.: %0.3f" % min_max_difference)
print("Root Mean Squared Error: %0.3f" % root_mean_squared_error)

Min. Median House Value: 0.000
Max. Median House Value: 461.000
Difference between Min. and Max.: 461.000


NameError: name 'root_mean_squared_error' is not defined